## First try of a* - DON'T LOOK THIS CODE

In [56]:
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappush, heappop
from collections import namedtuple
from typing import List
import heapq
import time
import heapq
from typing import List, Tuple, Callable
import numpy as np

In [57]:
# Define the Action type
Action = namedtuple('Action', ['pos1', 'pos2'])

# Define puzzle dimension
PUZZLE_DIM = 6

In [58]:
def manhattan_distance_2(state: np.ndarray, goal_state: np.ndarray) -> int:
    """Calculates the sum of Manhattan distances of tiles from their goal positions."""
    total_distance = 0
    for val in range(1, PUZZLE_DIM**2):
        current_pos = np.argwhere(state == val)[0]
        goal_pos = np.argwhere(goal_state == val)[0]
        total_distance += abs(current_pos[0] - goal_pos[0]) + abs(current_pos[1] - goal_pos[1])
    return total_distance

def available_actions(state: np.ndarray) -> List[Action]:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = []
    if x > 0:
        actions.append(Action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(Action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x, y + 1)))
    return actions

"""Apply the action to the state."""
def do_action(state: np.ndarray, action: Action) -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [59]:

def a_star(start_state: np.ndarray, goal_state: np.ndarray, heuristic_func,) -> List[np.ndarray]:
    """Solve the puzzle using A*."""
    open_set = []
    # Convert start state to tuple for immutability and comparison in sets
    start_state_tuple = tuple(start_state.flatten())
    goal_state_tuple = tuple(goal_state.flatten())

    distance = heuristic_func(start_state, goal_state)
    heappush(open_set, (distance, 0, start_state_tuple, []))

    visited = set()
    visited.add(start_state_tuple)

    while open_set:
        f, g, current_state_tuple, path = heappop(open_set)
        current_state = np.array(current_state_tuple).reshape((PUZZLE_DIM, PUZZLE_DIM))

        # Goal test
        if np.array_equal(current_state, goal_state):
            return path + [current_state]
        

        # Generate successors
        for act in available_actions(current_state):
            successor = do_action(current_state, act)
            successor_tuple = tuple(successor.flatten())
            if successor_tuple not in visited:
                

                new_path = path + [act]  # Copy and append current state current_state
                #new_cost = g + 1
                new_cost = g + 0.00 * len(new_path)#(new_priority, len(new_path), new_path, successor), heuristic_func(successor, goal_state)

                #heappush(open_set, (new_cost + heuristic_func(successor, goal_state), new_cost, successor_tuple, new_path))
                heapq.heappush(open_set, (new_cost, len(new_path), successor_tuple, new_path))

                visited.add(successor_tuple)

    return None  # No solution found


def a_star_better_priority(initial_state: np.ndarray, goal_state: np.ndarray, heuristic_func, length_criteria=0) -> list:
    """Solves the n^2-1 puzzle using A* search with the given heuristic function."""
    open_set = []
    distance=heuristic_func(initial_state, goal_state)
    heapq.heappush(open_set, (distance, 0, [], initial_state))

    visited = set()

    start_time = time.time()

    min_distance = float('inf')
    best_state = None

    while open_set:
        _, move_count, path, current_state = heapq.heappop(open_set) #f, g, current_state_tuple, path

        if np.array_equal(current_state, goal_state):
            return path, current_state

        state_tuple = tuple(current_state.flatten())
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        for act in available_actions(current_state):
            successor = do_action(current_state, act)
            new_path = path + [act]
            new_priority = heuristic_func(successor, goal_state) + length_criteria * len(new_path)
            heapq.heappush(open_set, (new_priority, len(new_path), new_path, successor))

            current_distance = heuristic_func(successor, goal_state)
            if current_distance < min_distance:
                min_distance = current_distance
                best_state = successor

        if time.time() - start_time >= 5:
            print(f"Current depth: {move_count}, queue size: {len(open_set)}, current min distance: {min_distance}")
            start_time = time.time()

    print(f"Best state with min distance: {min_distance}")
    print(best_state)
    return None

In [60]:
Goal_State = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

RANDOMIZE_STEPS = 1000 #100_000  # 100 steps
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
np.random.seed(42)  # For reproducibility

for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

print("Initial State:")
print(state)

# solution_path = a_star(state, Goal_State, manhattan_distance_2)
# print("\nSolution Steps:")
# print(f"Solution found in {len(solution_path)} moves.")
# for step in solution_path:
#     print(step)

Randomizing: 100%|██████████| 1000/1000 [00:00<00:00, 55624.42it/s]

Initial State:
[[ 3  8  5 15  2 18]
 [28  7 34  4 14 10]
 [ 1  6 20 16 31 22]
 [23 11 33  0 21 12]
 [13 19  9 32 30 17]
 [26 25 24 27 29 35]]


In [ ]:
solution, final_state = a_star_better_priority(state, Goal_State, manhattan_distance_2,0)
if solution:
    print(f"Solution found in {len(solution)} moves.")
    print(final_state)
    # print(solution)
else:
    print("No solution found.")


Current depth: 165, queue size: 2201, current min distance: 35
Current depth: 172, queue size: 4474, current min distance: 35
Current depth: 178, queue size: 6830, current min distance: 35
Current depth: 180, queue size: 9056, current min distance: 35
Current depth: 184, queue size: 11288, current min distance: 35
Current depth: 186, queue size: 13381, current min distance: 35
Current depth: 190, queue size: 15366, current min distance: 35
Current depth: 194, queue size: 17409, current min distance: 35
Current depth: 201, queue size: 19416, current min distance: 35
Current depth: 207, queue size: 21370, current min distance: 35
Current depth: 158, queue size: 23484, current min distance: 33
Current depth: 195, queue size: 25915, current min distance: 31
Current depth: 212, queue size: 27978, current min distance: 31
Current depth: 232, queue size: 30103, current min distance: 31
Current depth: 216, queue size: 32372, current min distance: 25
Current depth: 218, queue size: 34580, curre

In [ ]:
def greedy_search(start: np.ndarray, goal: np.ndarray) -> List[np.ndarray]:
    """Implementazione dell'algoritmo di ricerca Greedy usando la distanza di Manhattan."""
    # Coda di priorità per esplorare gli stati più promettenti (minore distanza)
    frontier = []
    heapq.heappush(frontier, (manhattan_distance_2(start, goal), start))  # (f, stato)

    visited = set()  # Insieme degli stati esplorati
    start_tuple = tuple(start.flatten())
    visited.add(start_tuple)  # Aggiungi lo stato iniziale esplorato come tupla

    parent_map = {start_tuple: None}  # Per ricostruire il percorso

    while frontier:
        # Prendi il nodo con la minima distanza di Manhattan
        _, current_state_tuple = heapq.heappop(frontier)
        current_state = np.array(current_state_tuple).reshape((PUZZLE_DIM, PUZZLE_DIM))

        # Se abbiamo raggiunto il goal, ricostruiamo il percorso
        if np.array_equal(current_state, goal):
            path = []
            while current_state is not None:
                path.append(current_state)
                current_state = parent_map[tuple(current_state.flatten())]
            return path[::-1]  # Ritorna il percorso dal start al goal

        # Esplora i successori
        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            new_state_tuple = tuple(new_state.flatten())
            if new_state_tuple not in visited:
                visited.add(new_state_tuple)  # Marca come esplorato
                heapq.heappush(frontier, (manhattan_distance_2(new_state, goal), new_state_tuple))
                parent_map[new_state_tuple] = current_state  # Salva il predecessore

    return None  # Nessuna soluzione trovata


In [ ]:
# solution_path=greedy_search(state, Goal_State)

# if solution_path:
#     print(f"Solution found in {len(solution_path)} moves.")
# else:
#     print("No solution found.")